In [3]:
import numpy as np 
import glob
import cv2
import pandas as pd

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
import os
from tensorflow.keras.applications.vgg16 import VGG16


  Using cached opencv_python-4.6.0.66-cp36-abi3-macosx_10_15_x86_64.whl (46.4 MB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
SIZE = 128
images = []
labels = []

for directory_path in glob.glob("X-DDD/*"):
    label = directory_path.split("\\")[-1]
    print(label)
    for img_path in glob.glob(os.path.join(directory_path, "*")):
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)       
        img = cv2.resize(img, (SIZE, SIZE))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        images.append(img)
        labels.append(label)

images = np.array(images)
labels = np.array(labels)

X-DDD/oxide scale of temperature system
X-DDD/slag inclusion
X-DDD/finishing roll printing
X-DDD/iron sheet ash
X-DDD/surface scratch
X-DDD/red iron
X-DDD/oxide scale of plate system


In [5]:
images = images / 255.0
VGG_model = VGG16(weights='imagenet', include_top=False, input_shape=(SIZE, SIZE, 3),pooling='avg')
for layer in VGG_model.layers:
    layer.trainable = False
VGG_model.summary()

2022-07-09 04:05:45.795907: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


58889256/58889256 [==============================] - 32s 1us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 128, 128, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 128, 128, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 64, 64, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 64, 64, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 64, 64, 128)       147584    
                                                             

In [6]:
feature_extractor = VGG_model.predict(images)
features = feature_extractor.reshape(feature_extractor.shape[0], -1)

43/43 [==============================] - 199s 5s/step


In [8]:
F_1 = pd.DataFrame(features)
F_1['label'] = labels

In [10]:
F_1.shape

(1360, 513)

In [11]:
F_1.to_csv("features_extraction_VGG16.csv")